In [25]:
# this is the testing area

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from stn import spatial_transformer_network as transformer
from tensorflow.keras.regularizers import l2
import tensorflow.keras.activations as ac
import numpy as np
import matplotlib.pyplot as plt
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2



In [27]:
train = ImageDataGenerator(rescale=1 / 255,validation_split=0.2)
test = ImageDataGenerator(rescale=1 / 255)

In [28]:
image_size=(250,250)

In [29]:
train_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",
                                          subset="training",
                                          
                                         )
valid_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",                                          
                                          subset="validation")

test_dataset = test.flow_from_directory("../database/sharpness1reduce/test",
                                        target_size=image_size,                                       
                                        class_mode="categorical")

Found 3282 images belonging to 49 classes.
Found 795 images belonging to 49 classes.
Found 1919 images belonging to 49 classes.


In [32]:
model = tf.keras.models.Sequential()
    
model.add(tf.keras.Input(shape=(250, 250, 3)))
model.add(tf.keras.layers.Conv2D(32,(7, 7),padding="same",activation="relu"))       
model.add(tf.keras.layers.Conv2D(32,(5, 5),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(32,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))
        
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))
        
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same",activation="relu")) 
model.add(tf.keras.layers.Conv2D(128,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))

model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))

model.add(tf.keras.layers.Flatten())



model.add(tf.keras.layers.Dense(1024,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1024,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1024,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(512,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(512,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))


        
model.add(tf.keras.layers.Dense(256,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(256,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
        
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(49, activation="softmax"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_75 (Conv2D)           (None, 250, 250, 32)      4736      
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 250, 250, 32)      25632     
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 248, 248, 32)      9248      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 124, 124, 64)      18496     
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 124, 124, 64)      36928     
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 122, 122, 64)     

In [33]:

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_dataset, steps_per_epoch=40,validation_data=valid_dataset, epochs=250,shuffle=True)


score = model.evaluate(test_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("echo this is the end ***")


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 40 steps, validate for 25 steps
Epoch 1/250
40/40 [==============================] - 13s 320ms/step - loss: 30.1435 - accuracy: 0.0269 - val_loss: 78.5657 - val_accuracy: 0.0063
Epoch 2/250
40/40 [==============================] - 10s 255ms/step - loss: 28.1265 - accuracy: 0.0308 - val_loss: 81.8599 - val_accuracy: 0.0478
Epoch 3/250
40/40 [==============================] - 10s 257ms/step - loss: 26.3417 - accuracy: 0.0570 - val_loss: 46.5419 - val_accuracy: 0.0289
Epoch 4/250
40/40 [==============================] - 10s 258ms/step - loss: 24.7032 - accuracy: 0.0711 - val_loss: 54.6742 - val_accuracy: 0.0629
Epoch 5/250
40/40 [==============================] - 10s 259ms/step - loss: 23.2493 - accuracy: 0.0898 - val_loss: 114.1117 - val_accuracy: 0.0314
Epoch 6/250
40/40 [==============================] - 10s 258ms/step - loss: 21.8691 - accuracy: 0.1125 - val_loss: 22.7094 - val_accuracy: 0.0780
Epoch 7/250
40/40 [============

Epoch 56/250
40/40 [==============================] - 10s 257ms/step - loss: 2.9759 - accuracy: 0.6555 - val_loss: 2.7739 - val_accuracy: 0.7044
Epoch 57/250
40/40 [==============================] - 10s 256ms/step - loss: 2.9242 - accuracy: 0.6469 - val_loss: 2.5331 - val_accuracy: 0.7673
Epoch 58/250
40/40 [==============================] - 10s 255ms/step - loss: 3.6296 - accuracy: 0.5482 - val_loss: 5.4942 - val_accuracy: 0.4642
Epoch 59/250
40/40 [==============================] - 10s 255ms/step - loss: 3.9655 - accuracy: 0.5782 - val_loss: 3.9490 - val_accuracy: 0.6013
Epoch 60/250
40/40 [==============================] - 10s 255ms/step - loss: 3.6695 - accuracy: 0.5924 - val_loss: 3.1281 - val_accuracy: 0.7409
Epoch 61/250
40/40 [==============================] - 10s 257ms/step - loss: 3.3776 - accuracy: 0.6344 - val_loss: 3.2767 - val_accuracy: 0.6264
Epoch 62/250
40/40 [==============================] - 10s 257ms/step - loss: 3.1881 - accuracy: 0.6586 - val_loss: 2.7934 - val_ac

40/40 [==============================] - 10s 257ms/step - loss: 3.1027 - accuracy: 0.7641 - val_loss: 3.5657 - val_accuracy: 0.8302
Epoch 113/250
40/40 [==============================] - 10s 257ms/step - loss: 3.0927 - accuracy: 0.7750 - val_loss: 2.8676 - val_accuracy: 0.8553
Epoch 114/250
40/40 [==============================] - 10s 255ms/step - loss: 3.1494 - accuracy: 0.7757 - val_loss: 8.3568 - val_accuracy: 0.8252
Epoch 115/250
40/40 [==============================] - 10s 257ms/step - loss: 3.1246 - accuracy: 0.7625 - val_loss: 2.7902 - val_accuracy: 0.8755
Epoch 116/250
40/40 [==============================] - 10s 255ms/step - loss: 3.0373 - accuracy: 0.7796 - val_loss: 4.0800 - val_accuracy: 0.8717
Epoch 117/250
40/40 [==============================] - 10s 257ms/step - loss: 2.9683 - accuracy: 0.7906 - val_loss: 10.4471 - val_accuracy: 0.4138
Epoch 118/250
40/40 [==============================] - 10s 255ms/step - loss: 2.9526 - accuracy: 0.7836 - val_loss: 21.4514 - val_accurac

40/40 [==============================] - 10s 257ms/step - loss: 2.9902 - accuracy: 0.8328 - val_loss: 3.3506 - val_accuracy: 0.8214
Epoch 169/250
40/40 [==============================] - 10s 257ms/step - loss: 3.0804 - accuracy: 0.8195 - val_loss: 2.6649 - val_accuracy: 0.9245
Epoch 170/250
40/40 [==============================] - 10s 257ms/step - loss: 2.9615 - accuracy: 0.8352 - val_loss: 2.7928 - val_accuracy: 0.8679
Epoch 171/250
40/40 [==============================] - 10s 255ms/step - loss: 2.9642 - accuracy: 0.8246 - val_loss: 2.6584 - val_accuracy: 0.9208
Epoch 172/250
40/40 [==============================] - 10s 255ms/step - loss: 2.9159 - accuracy: 0.8349 - val_loss: 2.6004 - val_accuracy: 0.9233
Epoch 173/250
40/40 [==============================] - 10s 255ms/step - loss: 2.8566 - accuracy: 0.8468 - val_loss: 2.6352 - val_accuracy: 0.9082
Epoch 174/250
40/40 [==============================] - 10s 257ms/step - loss: 2.9406 - accuracy: 0.8258 - val_loss: 2.7884 - val_accuracy:

40/40 [==============================] - 10s 257ms/step - loss: 2.8627 - accuracy: 0.8523 - val_loss: 2.5326 - val_accuracy: 0.9560
Epoch 225/250
40/40 [==============================] - 10s 255ms/step - loss: 2.7360 - accuracy: 0.8578 - val_loss: 2.5077 - val_accuracy: 0.9195
Epoch 226/250
40/40 [==============================] - 10s 257ms/step - loss: 2.8185 - accuracy: 0.8539 - val_loss: 2.5119 - val_accuracy: 0.9358
Epoch 227/250
40/40 [==============================] - 10s 257ms/step - loss: 2.7322 - accuracy: 0.8609 - val_loss: 2.4715 - val_accuracy: 0.9346
Epoch 228/250
40/40 [==============================] - 10s 257ms/step - loss: 2.7194 - accuracy: 0.8477 - val_loss: 2.4358 - val_accuracy: 0.9321
Epoch 229/250
40/40 [==============================] - 10s 257ms/step - loss: 2.7657 - accuracy: 0.8461 - val_loss: 2.4348 - val_accuracy: 0.9233
Epoch 230/250
40/40 [==============================] - 10s 257ms/step - loss: 2.6920 - accuracy: 0.8438 - val_loss: 2.3630 - val_accuracy: